# Working with raw KIDs data

## Authors
Zhiyuan Ma

## Learning Goals
* Learn the raw KIDs data format
* Plot the KIDs resonators and timestreams

## Keywords
KIDs data

## Summary
In this tutorial, we will download a raw KIDs data file, and visualize it.

1. Setup some example datafiles

In [5]:
from astropy.utils.data import download_file
kidsdata_filepath = download_file('https://drive.google.com/uc?export=download&id=1Oeq3somvNV3Xiva_2N_2jN5VXixUmrE9')
print(f"downloaded {kidsdata_filepath}")

downloaded /var/folders/zc/33kgh8vx3z37kpp6xf84bzvm0000gn/T/astropy-download-23188-e7e2vgms


In [8]:
from tolteca.datamodels.toltec import BasicObsData
bod = BasicObsData(kidsdata_filepath, open_=True)
print(bod)

ValueError: unable to parse meta from source FileLoc(/var/folders/zc/33kgh8vx3z37kpp6xf84bzvm0000gn/T/astropy-download-23188-e7e2vgms)